<a href="https://colab.research.google.com/github/pdh0184/Celine_project/blob/main/SIFT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9C%A0%EC%82%AC_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-gpu

In [ ]:
import pandas as pd
import base64
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow  # Colab에서 cv2.imshow() 대신 사용

df = pd.read_csv('/content/drive/MyDrive/개인 프로젝트 파일/Celine_project/CELINE_DATA.csv')
df.head()

In [ ]:

sift = cv2.SIFT_create()
img_list = []
features_list = []

for index, row in df.iterrows():
  try:
    image_str64 = row['이미지 데이터']
    image_data = base64.b64decode(image_str64)

    # 바이너리 데이터를 이미지로 변환
    img = np.array(Image.open(BytesIO(image_data)))
    img_list.append(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    keypoint , description = sift.detectAndCompute(img,None)
    features_list.append(description)
  except Exception as e:
    print(f"Error processing row {index}: {e}")
    features_list.append(np.zeros((128,128)))


In [ ]:
print(features_list[0].shape)
print(features_list[1].shape)
print(features_list[42].shape)


In [ ]:
print(features_list[0].shape)
print(features_list[1].shape)
print(features_list[42].shape)


In [ ]:
features_list = [np.mean(desc, axis=0) for desc in features_list if desc is not None]

features_array = np.array(features_list, dtype='float32')

In [ ]:
import faiss

index = faiss.IndexFlatL2(features_array.shape[1])  # 128차원 L2 거리 기반
index.add(features_array)  # 이미지 벡터 추가

# 이미지 간 유사도 계산
D, I = index.search(features_array, k=11)  # 각 이미지에 대해 가장 유사한 5개 검색
print("Distances:", D)
print("Indices:", I)


In [ ]:
import time

input_product = int(input("제품 번호 입력") )#원하는 이미지 선택 시 해당 되는 인덱스 번호가 있다고 가정

start_time = time.time()

i = 0

for index in I[input_product]:
  if i == 0: #처음 인덱스는 자기 자신이므로 제외한다
    i += 1
    continue
  print(df["제품명"].iloc[index])
  print(df["가격"].iloc[index])
  print(df["상세주소"].iloc[index])

  plt.imshow(img_list[index])
  plt.axis('off')  # 축 제거
  plt.show()
  i += 1

end_time = time.time()

# 소요 시간 계산
elapsed_time = end_time - start_time
print(f"총 연산 시간: {elapsed_time:.6f} seconds")
print(f"총 추천 제품 개수: {i-1}개")

